In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from tqdm import tqdm_notebook
# Census API Key
from config import api_key
c = Census(api_key, year=2017)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",'B15003_002E', 'B15003_003E', 
                          'B15003_004E', 'B15003_005E', 'B15003_006E', 'B15003_007E', 'B15003_008E',
                          'B15003_009E', 'B15003_010E', 'B15003_011E', 'B15003_012E', 'B15003_013E',
                          'B15003_014E', 'B15003_015E', 'B15003_016E', 'B15003_017E', 'B15003_018E',
                          'B15003_019E', 'B15003_020E', "B15003_021E", "B15003_022E", "B15003_023E",
                          "B15003_024E", "B15003_025E", 'B15003_001E', 'B12001_001E', 'B12001_003E',
                          'B12001_004E', 'B12001_007E', 'B12001_009E', 'B12001_010E', 'B12001_011E', 
                          'B12001_012E', 'B12001_013E', 'B12001_016E', 'B12001_018E', 'B12001_019E')
                          , {'for': 'state:*'})


# Convert to DataFrame
census_df = pd.DataFrame(census_data)

# Column Reordering
census_df = census_df.rename(columns={'B01003_001E': 'Population',
                                      'B01002_001E': 'Median Age',
                                      'B19013_001E': 'Household Income',
                                      'B19301_001E': 'Per Capita Income',
                                      'B17001_002E': 'Poverty Count',
                                      'NAME': 'State', 
                                      'state': 'State Number',
                                      'B15003_021E' : 'Associate\'s Degree Count',
                                      'B15003_022E' : 'Bachelor\'s Degree Count',
                                      'B15003_023E' : 'Master\'s Degree Count',
                                      'B15003_024E' : 'Professional School Degree Count',
                                      'B15003_025E' : 'Doctorate Degree Count',
                                      'B15003_001E' : 'Total educational attainment over 25',
                                      'B15003_002E' : 'No School',
                                      'B15003_003E' : 'Nursery school',
                                      'B15003_004E' : 'Kindergarten',
                                      'B15003_005E' : '1st grade',
                                      'B15003_006E' : '2nd grade',
                                      'B15003_007E' : '3rd grade',
                                      'B15003_008E' : '4th grade',
                                      'B15003_009E' : '5th grade',
                                      'B15003_010E' : '6th grade',
                                      'B15003_011E' : '7th grade',
                                      'B15003_012E' : '8th grade',
                                      'B15003_013E' : '9th grade',
                                      'B15003_014E' : '10th grade',
                                      'B15003_015E' : '11th grade',
                                      'B15003_016E' : '12th grade No Diploma',
                                      'B15003_017E' : 'HS Diploma',
                                      'B15003_018E' : 'GED',
                                      'B15003_019E' : 'Some College less than 1 year',
                                      'B15003_020E' : 'Some College More than 1 year',
                                      'B12001_001E' : 'Over 15 Marital Status',
                                      'B12001_003E' : 'Male Never Married',
                                      'B12001_004E' : 'Male Married',
                                      'B12001_007E' : 'Male Spouse Seperated',
                                      'B12001_009E' : 'Male Widowed',
                                      'B12001_010E' : 'Male Divorced',
                                      'B12001_012E' : 'Female Never Married',
                                      'B12001_013E' : 'Female Married',
                                      'B12001_016E' : 'Female Spouse Seperated',
                                      'B12001_018E' : 'Female Widowed',
                                      'B12001_019E' : 'Female Divorced'})

census_df["Poverty Rate"] = (100 * census_df["Poverty Count"].astype(int)
                            / census_df["Population"].astype(int))
census_df["Total College Degrees"] = (census_df['Associate\'s Degree Count'].astype(int)+
                             census_df['Bachelor\'s Degree Count'].astype(int)+
                             census_df['Master\'s Degree Count'].astype(int)+
                             census_df['Professional School Degree Count'].astype(int)+
                             census_df['Doctorate Degree Count'].astype(int))
census_df['Total No HS Diploma'] = (census_df['No School'].astype(int)+
                                    census_df['Nursery school'].astype(int)+
                                    census_df['Kindergarten'].astype(int)+
                                    census_df['1st grade'].astype(int)+
                                    census_df['2nd grade'].astype(int)+
                                    census_df['3rd grade'].astype(int)+
                                    census_df['4th grade'].astype(int)+
                                    census_df['5th grade'].astype(int)+
                                    census_df['6th grade'].astype(int)+
                                    census_df['7th grade'].astype(int)+
                                    census_df['8th grade'].astype(int)+
                                    census_df['9th grade'].astype(int)+
                                    census_df['10th grade'].astype(int)+
                                    census_df['11th grade'].astype(int)+
                                    census_df['12th grade No Diploma'].astype(int))
census_df['Total HS Diploma'] = (census_df['HS Diploma'].astype(int)+
                                    census_df['GED'].astype(int))

census_df['Total Some College'] = (census_df['Some College less than 1 year'].astype(int)+
                                    census_df['Some College More than 1 year'].astype(int))

census_df['Education Check'] = (census_df['Total College Degrees'].astype(int)+
                             census_df['Total No HS Diploma'].astype(int)+
                             census_df['Total HS Diploma'].astype(int)+
                             census_df['Total Some College'].astype(int))

census_df["Never Married"] = (census_df['Male Never Married'].astype(int)+
                             census_df['Female Never Married'].astype(int))


census_df["Married"] = (census_df['Male Married'].astype(int)+
                        census_df['Female Married'].astype(int))
                        
census_df["Widowed"] = (census_df['Male Widowed'].astype(int)+
                             census_df['Female Widowed'].astype(int))

census_df["Divorced"] = (census_df['Male Divorced'].astype(int)+
                             census_df['Female Divorced'].astype(int))

census_df['Married Check'] = (census_df['Married'].astype(int)+
                             census_df['Widowed'].astype(int)+
                             census_df['Divorced'].astype(int)+
                             census_df["Never Married"].astype(int))

census_df['College Degree Rate'] = (census_df['Total College Degrees']/census_df['Total educational attainment over 25']*100)

census_df['No High School Diploma Rate'] = (census_df['Total No HS Diploma']/census_df['Total educational attainment over 25']*100)

census_df['HS Diploma Rate'] = (census_df['Total HS Diploma']/census_df['Total educational attainment over 25']*100)

census_df['Some College Rate'] = (census_df['Total Some College']/census_df['Total educational attainment over 25']*100)

census_df['Married Rate'] = (census_df['Married']/census_df['Over 15 Marital Status']*100)

census_df['Widowed Rate'] = (census_df['Widowed']/census_df['Over 15 Marital Status']*100)

census_df['Divorced Rate'] = (census_df['Divorced']/census_df['Over 15 Marital Status']*100)

census_df['Never Married Rate'] = (census_df['Never Married']/census_df['Over 15 Marital Status']*100)

# Final DataFrame
census_df = census_df[['State', 'Population', 'Median Age', 'Household Income',
                       'Per Capita Income', 'Poverty Count', 'Poverty Rate', 'Total No HS Diploma', 'No High School Diploma Rate',
                       'Total HS Diploma', 'HS Diploma Rate', 'Total Some College', 'Some College Rate', 'Total College Degrees',
                       'College Degree Rate', 'Total educational attainment over 25','Never Married', 'Never Married Rate', 'Married','Married Rate', 'Widowed',
                       'Widowed Rate', 'Divorced', 'Divorced Rate', 'Over 15 Marital Status']]
# Visualize
print(len(census_df))

52


In [3]:
census_df.loc[census_df['Household Income'] < 0, 'Household Income'] = 0
census_df.loc[census_df['Median Age'] < 0, 'Median Age'] = 0
census_df.loc[census_df['Per Capita Income'] < 0, 'Per Capita Income'] = 0

In [4]:
census_df=census_df.sort_values(by='State', ascending=True)
census_df = census_df.reset_index(drop=True)
census_df.head()

,State,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Total No HS Diploma,No High School Diploma Rate,Total HS Diploma,...,Total educational attainment over 25,Never Married,Never Married Rate,Married,Married Rate,Widowed,Widowed Rate,Divorced,Divorced Rate,Over 15 Marital Status
0,Alabama,4850771.0,38.7,46472.0,25746.0,849699.0,17.516782,481059,14.681486,1012551,...,3276637.0,1197758,30.389444,1979275,50.218046,280457,7.115738,483872,12.276771,3941362.0
1,Alaska,738565.0,33.9,76114.0,35065.0,73380.0,9.935483,36177,7.609130,131293,...,475442.0,198336,34.069394,294776,50.635486,21707,3.728745,67334,11.566375,582153.0
2,Arizona,6809946.0,37.2,53510.0,27964.0,1128046.0,16.564683,609649,13.499233,1091339,...,4516175.0,1807995,33.101270,2681012,49.084705,301048,5.511670,671956,12.302355,5462011.0
3,Arkansas,2977944.0,37.9,43813.0,24426.0,524211.0,17.603118,286117,14.408366,681486,...,1985770.0,660401,27.611923,1253607,52.414367,165004,6.898957,312712,13.074753,2391724.0
4,California,38982847.0,36.1,67169.0,33128.0,5773408.0,14.810124,4543530,17.508234,5345542,...,25950818.0,11553117,36.770353,15330135,48.791549,1568495,4.992083,2967905,9.446015,31419652.0


In [5]:
census_df.describe()

,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Total No HS Diploma,No High School Diploma Rate,Total HS Diploma,HS Diploma Rate,...,Total educational attainment over 25,Never Married,Never Married Rate,Married,Married Rate,Widowed,Widowed Rate,Divorced,Divorced Rate,Over 15 Marital Status
count,5.200000e+01,52.000000,52.000000,52.000000,5.200000e+01,52.000000,5.200000e+01,52.000000,5.200000e+01,52.000000,...,5.200000e+01,5.200000e+01,52.000000,5.200000e+01,52.000000,5.200000e+01,52.000000,5.200000e+01,52.000000,5.200000e+01
mean,6.239872e+06,38.155769,57496.826923,30641.807692,9.075686e+05,14.292235,5.392833e+05,11.376476,1.149096e+06,28.330204,...,4.205047e+06,1.677281e+06,32.325377,2.532181e+06,50.448013,2.956408e+05,5.877746,5.501656e+05,11.348863,5.055268e+06
std,7.124336e+06,2.433138,11115.944598,5755.331373,1.075230e+06,5.168543,7.758502e+05,3.479304,1.169081e+06,4.109970,...,4.764035e+06,2.036033e+06,4.401799,2.833815e+06,3.999292,3.177745e+05,0.816777,5.816165e+05,1.438500,5.745040e+06
min,5.832000e+05,30.500000,19775.000000,12081.000000,6.339800e+04,7.843988,2.813000e+04,7.024104,8.411900e+04,17.603899,...,3.886170e+05,1.260330e+05,26.810051,1.748200e+05,30.676418,2.170700e+04,3.709454,5.175700e+04,8.502416,4.667770e+05
25%,1.791976e+06,36.725000,50862.500000,27298.000000,2.320825e+05,11.172224,1.102482e+05,8.786896,3.246050e+05,26.199858,...,1.191710e+06,4.082848e+05,29.659546,7.664922e+05,49.243741,8.013400e+04,5.504178,1.555168e+05,10.280626,1.448762e+06
50%,4.224752e+06,38.250000,56344.500000,29733.000000,6.124285e+05,14.017835,3.244090e+05,10.451235,7.977190e+05,27.965165,...,2.892076e+06,1.076720e+06,31.818740,1.710600e+06,51.048285,2.048860e+05,5.959182,4.287735e+05,11.548784,3.446218e+06
75%,6.899951e+06,39.450000,65358.250000,33550.250000,9.717790e+05,15.833668,6.066272e+05,13.559146,1.399394e+06,31.004527,...,4.753916e+06,1.873650e+06,33.902112,2.837208e+06,52.425913,3.232508e+05,6.371375,6.674770e+05,12.412914,5.698155e+06
max,3.898285e+07,44.300000,78916.000000,50832.000000,5.773408e+06,44.486494,4.543530e+06,25.331365,5.345542e+06,40.559959,...,2.595082e+07,1.155312e+07,55.801181,1.533014e+07,57.616323,1.568495e+06,7.787737,2.967905e+06,13.894467,3.141965e+07


In [6]:
census_df.to_csv('Project1_State_Data.csv',encoding="utf-8", index=False)